## 感知机 Perceptron
给定输入x，权重w和偏移b，感知机输出：
$$
o = \sigma(<w, x> + b), \sigma(x) = 
\begin{cases}
    1, if x > 0\\
    -1, otherwise
\end{cases}
$$
**二分类**


* 训练感知机——等价于使用batch_size=1的梯度下降，使用损失函数：
$$
\mathcal{l}(y, x, w) = max(0, -y<w, x>)
$$

```
initialize w = 0, b = 0
repeat
    if y[i](<w, x[i]> + b) then
        w = w + y[i]x[i]
        b = b + y[i]
    end if
until all classified correctly
```

* 收敛定理
    * 数据在半径r内
    * 余量rho分类两类
        $$
            y(x^T w + b) \ge \rho
        $$
        对于$||w||^2 + b^2 \le 1$
    * 感知机保证在$\frac{r^2 + 1}{\rho^2}$步后收敛

* XOR问题
感知机不能拟合XOR函数，只能产生线性分割面(需要核函数用核支持向量机)————导致第一次AI寒冬

## 多层感知机MLP
*  学习XOR——使用两层感知机

* 单隐藏层——隐藏层大小是一个超参数
#### 单分类
* 输入$x \in R^n$
* 隐藏层$W_1 \in R^{m \times n}, b_1 \in R^m$
* 输出层$w_2 \in R^m, b_2 \in R$
    $$
    h = \sigma(W_1 x + b)\\
    o = w_2^Th + b_2
    $$
    其中sigma是按元素的激活函数
![Hidden Layer](./imgs/hidden-layer.png)

* 为什么需要激活函数
如果没有激活函数，12, 23层整体作用结果是一个线性函数，那整体还是一个线性模型，和一层没有区别好的吧


### 激活函数
* Sigmoid激活函数
    * 将输入投影到(1, 0)，是一个软的激活函数
    $$
    \sigma(x) = 
    \begin{cases}
    1, if~x > 0\\
    0, other wise
    \end{cases}
    $$
    
    $$
    sigmoid(x) = \frac{1}{1 + exp(-x)}
    $$
    ![sigmoid](./imgs/sigmoid.png)

* tanh激活函数
    * 将输入投影到(-1, 1)
    $$
    tanh(x) = \frac{1 - exp(-2x)}{1 + exp(-2x)}
    $$
    ![tanh](./imgs/tanh.png)
* ReLU激活函数
    * ReLU: rectified linear unit
    $$
    ReLU(x) = max(x, 0)
    $$
    ![ReLU](./imgs/ReLU.png)

**注意，指数运算是很贵的，算一次指数几乎相当于**

#### 多分类
$y_1, y_2, ..., y_k = softmax(o_1, o_2, ..., o_k)$
加个隐藏层然后再做softmax回归
* 输入$x \in R^n$
* 隐藏层$W_1 \in R^{m \times n}, b_1 \in R^m$
* 输出层$W_2 \in R^{m \times k}, b_2 \in R^k$
    $$
    h = \sigma(W_1x + b_1)\\
    o = W_2^Th + b_2\\
    y = softmax(o)
    $$

可以做多隐藏层，超参数：隐藏层数&每层隐藏层的大小

注意提取特征需要慢慢压缩


## MLP代码实现
#### 手搓

In [7]:
import torch
from torch import nn
from d2l import torch as d2l

batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

In [8]:
# 实现一个具有单隐藏层的MLP，包含256个隐藏单元

num_inputs, num_outputs, num_hiddens = 28 * 28, 10, 256

W1 = nn.Parameter(
    torch.randn(num_inputs,num_hiddens, requires_grad=True)
)
b1 = nn.Parameter(
    torch.zeros(num_hiddens, requires_grad=True)
)
W2 = nn.Parameter(
    torch.randn(num_hiddens, num_outputs, requires_grad=True)
)
b2 = nn.Parameter(
    torch.zeros(num_outputs, requires_grad=True)
)

params = [W1, b1, W2, b2]

In [9]:
# 实现relu几乎函数
def relu(X):
    a = torch.zeros_like(X)
    return torch.max(X, a)

In [10]:
# 实现模型
def net(X):
    X = X.reshape((-1, num_inputs))
    H = relu(X @ W1 + b1)
    return (H @ W2 + b2)

loss = nn.CrossEntropyLoss()

In [12]:
# 训练过程同前面的spftmax回归的训练过程

num_epochs, lr = 10, 0.1
updater = torch.optim.SGD(params, lr=lr)
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, updater)

AttributeError: module 'd2l.torch' has no attribute 'train_ch3'